<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/subredditValidation/uofmn_test_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
COLLEGE_SUBREDDIT_NAME = "uofmn"
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models/" + COLLEGE_SUBREDDIT_NAME
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['train_post_set.pickle', 'val_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'checkpoints-uofmn']


In [2]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/uofmn


In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [4]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/uofmn


In [5]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=

In [8]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [10]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [11]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/uofmn


In [12]:
val_post_path = os.path.join(GOOGLE_DRIVE_PATH, "val_post_set.pickle")
val_com_path = os.path.join(GOOGLE_DRIVE_PATH, "val_com_set.pickle")

In [13]:
print(val_post_path)

drive/My Drive/448 Group: Jedi Order/models/uofmn/val_post_set.pickle


In [14]:
# posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
val_posts = pickle.load(open(val_post_path, "rb"))
# posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
# comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
val_comments = pickle.load(open(val_com_path, "rb"))
# comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [15]:
for pt in val_posts:
  print(pt)

<bos> Hey so I have just received my acceptance to attend Carlson school of management on exchange (I am from Australia) in the spring semester next year. I have the options of either middlebrook, centennial or university village for my accommodation. I am quite a social person and wouldn't mind attending regular parties etc. I was just curious as to what would be the best option of those 3 for what I am after. Also how do the meal plans and that work? Plus any further general advice I should be aware of. I want to fully immerse myself in the whole college life/atmosphere. Cheers  <eos> 
<bos> I am considering taking Animal Diversity or Human Physiology. What are your opinions or experiences with the course, the difficulty, usefulness of knowledge (I am considering a future career in health or research), and the fairness of grading. Also, for cell biology BIOL4004 what professors did you have and what are your thoughts on the criteria I listed above. Any help would be greatly appreciat

In [16]:
print(val_posts)

["<bos> Hey so I have just received my acceptance to attend Carlson school of management on exchange (I am from Australia) in the spring semester next year. I have the options of either middlebrook, centennial or university village for my accommodation. I am quite a social person and wouldn't mind attending regular parties etc. I was just curious as to what would be the best option of those 3 for what I am after. Also how do the meal plans and that work? Plus any further general advice I should be aware of. I want to fully immerse myself in the whole college life/atmosphere. Cheers  <eos> ", '<bos> I am considering taking Animal Diversity or Human Physiology. What are your opinions or experiences with the course, the difficulty, usefulness of knowledge (I am considering a future career in health or research), and the fairness of grading. Also, for cell biology BIOL4004 what professors did you have and what are your thoughts on the criteria I listed above. Any help would be greatly appr

Load the trained model

In [17]:
model_file_path = "checkpoints-" + COLLEGE_SUBREDDIT_NAME

In [18]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, model_file_path)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [19]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

^^^^^^run if needed 

In [20]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
# post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_in = "Hey everyone! For those of you who took BIOMI 2900, or General Microbiology, did you take the discussion? If so, did it help at all? Trying to decide if I should or not. I'm not a microbiology major but I'm in a microbio lab at the moment so some of it does make sense already. Thanks!"
post_with_bos = post_in + " TL;DR"
encoded_prompt = tokenizer(post_in, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
)

generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [21]:
# print(generated_sequences)
for sent in generated_sequences:
  print(sent.replace(post_in, '').replace('TL;DR', ''))

   The professor at the end of the session is an awesome dude. The conversation you hear is really interesting. He was a good fellow. He was very helpful and helpful in the way I've said in his class, he was very knowledgeable, helpful, and helpful, and he made an important point of saying that a professor may not be good at
   I took the discussion and didn't have the right questions to answer. There was never any "major" question at all. Most labs have two questions that we can talk about, but I've never felt the need to go down the list of questions. Just a note though. I didn't have any sort of "major". I took one
   If you had taken BIOMI 3000, which did not take any chemistry class, I'd take it. I think you've seen the discussion. It's in my opinion it was more of an intellectual discussion about something you would want to do. If it did help, I would definitely recommend it. I did chemistry while I was a freshman


In [22]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=6b9662805fa8e7abe64f115292eddd1a48c6a989a6487d0efd3ba390814339f0
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [23]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [24]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  # post_with_bos = "<bos>" + post + " TL;DR"
  post_with_bos = post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [25]:
val_len = len(val_comments)
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(val_comments):
  val_dataset[cnt]['reference_summary'] = item.replace("<bos>", "").replace("<eos>", "")


print(val_dataset[1]['reference_summary'])

 I'm in Animal Diversity right now, but I've never taken Physiology so I can't make any comparisons for you. Animal diversity is a bit more work than a 2 credit course, but if you can memorize well, then you'll find it very easy. Tests are very straightforward; no multiple choice. Part of it is a lab practical, where you look at something and there's a pin stuck in a structure that you need to identify. Other questions are short answer that are about things that you straight memorize or about things that are about evolution or about a certain species. Personally, I love the course even with its heavier study load. I think the grading is very fair and I'm learning things that are important to my future with healthcare.  


In [26]:
for cnt, post in enumerate(val_posts):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [27]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: This suggests rock-climbing is free at St. Paul but not on East Bank. Is that true? What are the differences between the two? [url link]    I would definitely advise everyone to stop trying to walk to the West Bank after dark. The West Bank is free. The East Bank is the same, and if you're a guy, you are on your own. I took both. I took it with Normandale and he was fantastic. I took it at the U. He's super chill though, and I
Reference COMMENTS: I got super into rock climbing this past school year and would say that the Minneapolis wall is definitely worth it. If you are not completely sold on climbing yet, you can rent the shoes there, which helps to ease into it, rather than investing even more. If you already own shoes the St. Paul gym is worth a visit, but its is much smaller overall, and you are not allowed to top rope when you go. Therefore if you are interested in top roping, Minneapolis is for sure the way to go. The bouldering wall is about 2 feet shorter 

In [29]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

Overall ROUGE-1 scores: {'recall': 0.2555798054994514, 'precision': 0.19503013860022186, 'fmeasure': 0.2061306638944049}
Overall ROUGE-2 scores: {'recall': 0.0334091191549266, 'precision': 0.023365290427883243, 'fmeasure': 0.024832839238402193}
Overall ROUGE-L scores: {'recall': 0.22602812343087528, 'precision': 0.17166266352984527, 'fmeasure': 0.181379155874861}


**Calculating the sentiment analysis accuracy**

In [30]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.1 MB/s eta 0:00:00


In [31]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [32]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [33]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  # print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  # print(sent_gt)
  # print(gt_label)
  # print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  # print(sent_generated_comment)
  # print(generated_comment_label)
  # if idx == 15:
  #   break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

The number of generated comments that have aligned sentiments: 800
The number of posts in the validation set: 809
The accuracy of sentiments of the generated comments: 0.9888751545117429


**Readability validation test**

Automated readability index

In [34]:
!pip3 install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.6 MB/s eta 0:00:00


Flesch-Kincaid Grade Level: The Flesch-Kincaid Grade Level measures the approximate grade level of a text, based on its sentence length and average number of syllables per word.

In [35]:
import textstat

In [36]:
grade_level = 0.0
for idx in range(val_len):
  sc = textstat.flesch_kincaid_grade(val_dataset[idx]['generated_text'])
  # print(sc)
  grade_level += sc

grade_level /= val_len
print("Flesch-Kincaid Grade Level:", grade_level)

Flesch-Kincaid Grade Level: 5.978615574783679


Gunning Fog Index: The Gunning Fog Index measures the approximate years of education needed to understand a text, based on its sentence length and percentage of complex words.

In [37]:
fog_index = 0.0
for idx in range(val_len):
  sc = textstat.gunning_fog(val_dataset[idx]['generated_text'])
  # print(sc)
  fog_index += sc

fog_index /= val_len
print("Gunning Fog Index:", fog_index)

Gunning Fog Index: 8.124474660074169


Automated Readability Index (ARI): The Automated Readability Index measures the approximate grade level of a text, based on its average number of characters per word and average number of words per sentence.



In [38]:
ari = 0.0
for idx in range(val_len):
  sc = textstat.automated_readability_index(val_dataset[idx]['generated_text'])
  # print(sc)
  ari += sc

ari /= val_len
print("Automated Readability Index:", ari)

Automated Readability Index: 6.420024721878863
